In [8]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")



C:\Users\Dhruv\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dhruv\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [19]:
nlp = pipeline("ner", model=model, tokenizer=tokenizer)
with open(r"C:\Users\Dhruv\Desktop\entity_extraction\articles\article_24.txt", "r", encoding = "utf-8") as f:
  example = f.read()

# print(nlp)

ner_results = nlp(example)

# print(len(ner_results))

for entity in ner_results:
    print(entity)

for i, entity in enumerate(ner_results):
    if entity['word'][:2] == '##':
        entity['word'] = entity['word'][2:]
        
deleted = []

for i, entity in enumerate(ner_results):
    if(entity['entity'] == 'B-ORG' or entity['entity'] == 'B-PER' or entity['entity'] == 'B-MISC' or entity['entity'] == 'B-LOC' ):
        start = i
    if(entity['entity'] == 'I-ORG' 
       or entity['entity'] == 'I-PER' 
       or entity['entity'] == 'I-MISC' 
       or entity['entity'] == 'I-LOC'):
        deleted.append(i)
        ner_results[start]['word'] = ner_results[start]['word'] + entity['word']
    
print(deleted)

new_ner_results = [ner_results[i] for i in range(len(ner_results)) if not (i in deleted)]
        
for entity in new_ner_results:
    print(entity)



{'entity': 'I-ORG', 'score': 0.8245272, 'index': 3, 'word': 'DE', 'start': 4, 'end': 6}
{'entity': 'B-ORG', 'score': 0.52168685, 'index': 14, 'word': 'Dal', 'start': 33, 'end': 36}
{'entity': 'I-ORG', 'score': 0.9821968, 'index': 15, 'word': '##al', 'start': 36, 'end': 38}
{'entity': 'I-ORG', 'score': 0.9644283, 'index': 16, 'word': 'Street', 'start': 39, 'end': 45}
{'entity': 'B-MISC', 'score': 0.78110546, 'index': 45, 'word': 'IP', 'start': 184, 'end': 186}
{'entity': 'B-ORG', 'score': 0.99638015, 'index': 92, 'word': 'L', 'start': 369, 'end': 370}
{'entity': 'I-ORG', 'score': 0.9623303, 'index': 93, 'word': '##IC', 'start': 370, 'end': 372}
{'entity': 'B-LOC', 'score': 0.99944204, 'index': 163, 'word': 'India', 'start': 692, 'end': 697}
{'entity': 'B-PER', 'score': 0.9994803, 'index': 202, 'word': 'Nik', 'start': 851, 'end': 854}
{'entity': 'I-PER', 'score': 0.9812785, 'index': 203, 'word': '##hil', 'start': 854, 'end': 857}
{'entity': 'I-PER', 'score': 0.99971265, 'index': 204, 'wo

In [83]:
names = []
company = []
places = []

for entity in new_ner_results:
  if(entity['entity'] == 'B-PER' and names.count(entity['word']) == 0):
    names.append(entity['word'])
  if(entity['entity'] == 'B-ORG' and company.count(entity['word']) == 0):
    company.append(entity['word'])
  if(entity['entity'] == 'B-LOC' and places.count(entity['word']) == 0):
    places.append(entity['word'])
    
print(names)
print(company)
print(places)

['Ra', 'hulBhat', 'Ka', 'pilKaul']
['IndiGo', 'BSE', 'CAPAIndia', 'In', 'diGo']
[]


In [84]:
import openpyxl

wb = openpyxl.load_workbook(r'C:\Users\Dhruv\Desktop\entity_extraction\ner-bert\entities-bert.xlsx')
sheet = wb.active

s_name = "[" + ", ".join(name for name in names) + "]"
s_places = "[" + ", ".join(place for place in places) + "]" 
s_company = "[" + ", ".join(co for co in company) + "]" 

row = 4

sheet.cell(row = row, column = 1).value = s_name
sheet.cell(row = row, column = 2).value = s_company
sheet.cell(row = row, column = 3).value = s_places



wb.save(r"C:\Users\Dhruv\Desktop\entity_extraction\ner-bert\entities-bert.xlsx")
